In [1]:
import torch

In [2]:
torch.cuda.is_available()

True

In [3]:
import re
import os
from tqdm import tqdm
import nltk
import pickle
import re
from pprint import pprint


In [4]:
txt = open('./data_tmp/train(12809).txt', 'r', encoding='utf-8').read()
txt = txt.split('\n\n')
srcs = pickle.load(open('./data_tmp/train_tar_sts.pkl','rb'))
train_input = []
train_output = []
for para, src in tqdm(zip(txt,srcs)):
    sentences = para.split('\n')
    st1 = sentences[1]
    train_input.append(src)
    train_output.append('[CLS] '+st1+' [SEP]')

12801it [00:00, 716906.59it/s]


In [5]:
print(srcs[:5])

['[CLS] Stage 2 ulcers : On sacrum and scrotum . [SEP]', '[CLS] Labs were significant for day(s) - dimer 6218 , troponin 0 . 05 , creatinine 1 . 4 , Hco3 20 and wbc 11 . 3 . [SEP]', '[CLS] They drove down from Country 6607 to Hospital1 18 Emergency Department for evaluation . [SEP]', '[CLS] Serum toxicology screens were negative for benzodiazepenes , barbituates and tricyclic antidepressants . [SEP]', '[CLS] Known surname was admitted to the Neurosurgery service after a study revealed a break in her [unused4] catheter at the level of her neck . [SEP]']


In [6]:
def to_sentence(sentence, cudic):
    sentence = sentence[2:]
    text = re.sub('\[[^\[\]]*\]', '', sentence)
    pairs = re.findall('[^\[\] ]+\[[^\[\]]+\]', sentence)
    for pair in pairs:
        pair = re.split('[\[\]]', pair)
        cudic[pair[0]] = pair[1]
    words = nltk.word_tokenize(text)
    for wid, word in enumerate(words):
        if word in cudic.keys():
            words[wid] = cudic[word]
    return ' '.join(words), cudic

In [7]:
txt = open('./data_tmp/test(2030).txt', 'r', encoding='utf-8').read()
txt = txt.split('\n\n')
srcs = pickle.load(open('./data_tmp/test_tar_sts.pkl','rb'))
test_input = []
test_output = []
test_tars = []
cudics = []
for para, src in tqdm(zip(txt, srcs)):
    sentences = para.split('\n')
    test_input.append(src)
    cudic = {}
    test_tar = []
    
    for i, st in enumerate(sentences):
        if i==0:
            continue
        else:
            st_, cu = to_sentence(st,cudic)
            cudic.update(cu)
            test_tar.append(st_)
            if i==2:
                test_output.append(st_)
    test_tars.append(test_tar)
    cudics.append(cudic)

                

2031it [00:01, 1799.89it/s]


In [8]:
print(test_output[1])

She was intubated and was waken up after 10-22 minutes of unresponsive pulse arrest .


In [9]:
pickle.dump(cudics,open('./data_tmp/test_cudics.pkl','wb'))
pickle.dump(test_tars,open('./data_tmp/test_tars.pkl','wb'))


In [10]:
print(train_input[:5])
print(train_output[:5])
print(test_input[:5])
print(test_output[:5])
print(cudics[:5])


['[CLS] Stage 2 ulcers : On sacrum and scrotum . [SEP]', '[CLS] Labs were significant for day(s) - dimer 6218 , troponin 0 . 05 , creatinine 1 . 4 , Hco3 20 and wbc 11 . 3 . [SEP]', '[CLS] They drove down from Country 6607 to Hospital1 18 Emergency Department for evaluation . [SEP]', '[CLS] Serum toxicology screens were negative for benzodiazepenes , barbituates and tricyclic antidepressants . [SEP]', '[CLS] Known surname was admitted to the Neurosurgery service after a study revealed a break in her [unused4] catheter at the level of her neck . [SEP]']
['[CLS] Stage 2 break in skin : On sacrum and scrotum . [SEP]', '[CLS] Labs were significant for d-dimer 6218 , troponin 0.05 , creatinine 1.4 , Bicarbonate 20 and white blood cells 11.3 . [SEP]', '[CLS] They drove down from Country 6607 to Hospital1 18 Emergency Department for evaluation . [SEP]', '[CLS] Serum toxicology test were negative for benzodiazepenes , barbituates and tricyclic antidepressants . [SEP]', '[CLS] Patient was admit

In [11]:

for i,(u,v) in enumerate(zip(test_input,test_output)):
    if 'patient is a 89 year old' in v:
        print(u)
        print(v)
        print(i)

[CLS] [unused4] is a 89 years old male with sick sinus syndrome [unused4] extension [unused4] pacer , admitted to Neurosurgery with [unused4] with acute breathing distress extension [unused4] on the floor . [SEP]
patient is a 89 year old male with irregular heart rhythms after pacer , admitted to Neurosurgery with brain blood collects with acute respiratory distress/ wheezing on the floor .
8


In [12]:
with open("./data_tmp/train_input.pkl", 'wb') as f:
    pickle.dump(train_input,f)
with open("./data_tmp/train_output.pkl", 'wb') as f:
    pickle.dump(train_output,f)
with open("./data_tmp/test_input.pkl", 'wb') as f:
    pickle.dump(test_input,f)
with open("./data_tmp/test_output.pkl", 'wb') as f:
    pickle.dump(test_output,f)



In [13]:
from transformers import BertTokenizer
from importlib import import_module
import random
from keras.preprocessing.sequence import pad_sequences
model_name = 'bert'
x = import_module('models.' + model_name)
config = x.Config(16)
tokenizer = config.tokenizer
# tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case = False)

def tokenize(input_data, output_data):
    ids = []
    labels = []
    masks = []
    for i in tqdm(range(len(input_data))):
#         encoded_dict = tokenizer.encode_plus(
#                                 input_data[i],                      # Sentence to encode.
#                                 add_special_tokens = False, # Add '[CLS]' and '[SEP]'
#                                 max_length = 64,           # Pad & truncate all sentences.
#                                 pad_to_max_length = True,
#                                 return_attention_mask = True,   # Construct attn. masks.
#                                 return_tensors = 'pt',     # Return pytorch tensors.
#                            )
#         words = nltk.word_tokenize(input_data[i])
#         tokenized_sentence = []
#         for word in words:
#             tokenized_word = tokenizer.tokenize(word)
#             tokenized_sentence.extend(tokenized_word)
#         ids.append(tokenized_sentence)
        
#         words = nltk.word_tokenize(output_data[i])
#         tokenized_sentence = []
#         for word in words:
#             tokenized_word = tokenizer.tokenize(word)
#             tokenized_sentence.extend(tokenized_word)
#         labels.append(tokenized_sentence)
        words = tokenizer.tokenize(input_data[i])
        ids.append(words)
        words = tokenizer.tokenize(output_data[i])
        labels.append(words)
        
        
    
    ids = pad_sequences([tokenizer.convert_tokens_to_ids(txt) for txt in ids],
                          maxlen=64, dtype="long", value=0,
                              truncating="post", padding="post")
    labels = pad_sequences([tokenizer.convert_tokens_to_ids(txt) for txt in labels],
                          maxlen=64, dtype="long", value=0,
                              truncating="post", padding="post")
    
        
    return ids, labels
    
#         ids.append(encoded_dict['input_ids'])
#         masks.append(encoded_dict['attention_mask'])
#         encoded_dict = tokenizer.encode_plus(
#                                 output_data[i],add_special_tokens = False, max_length = 64, pad_to_max_length = True, return_attention_mask = True,
#                                 return_tensors = 'pt',)
#         labels.append(encoded_dict['input_ids'])
#     return ids, labels, masks
    
# train_ids, train_labels, train_mask = tokenize(train_input, train_output)
# test_ids, test_labels, test_mask = tokenize(test_input, test_output)
train_src, train_tar = tokenize(train_input, train_output)
test_src, test_tar = tokenize(test_input, test_output)
train_src_masks = [[float(i != 0.0) for i in ii] for ii in train_src]
train_tar_masks = [[float(i != 0.0) for i in ii] for ii in train_tar]
test_src_masks = [[float(i != 0.0) for i in ii] for ii in test_src]
test_tar_masks = [[float(i != 0.0) for i in ii] for ii in test_tar]




Using TensorFlow backend.
100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2031/2031 [00:00<00:00, 2132.69it/s]


In [14]:

pickle.dump(train_src, open("./data_tmp/train_src.pkl",'wb'))
pickle.dump(train_tar, open("./data_tmp/train_tar.pkl",'wb'))
pickle.dump(train_src_masks, open("./data_tmp/train_src_masks.pkl",'wb'))
pickle.dump(train_tar_masks, open("./data_tmp/train_tar_masks.pkl",'wb'))


pickle.dump(test_src, open("./data_tmp/test_src.pkl",'wb'))
pickle.dump(test_tar, open("./data_tmp/test_tar.pkl",'wb'))
pickle.dump(test_src_masks, open("./data_tmp/test_src_masks.pkl",'wb'))
pickle.dump(test_tar_masks, open("./data_tmp/test_tar_masks.pkl",'wb'))

